# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Setup the Environment

Run the next code cell to setup the environment.  This line might take a few minutes to run!

In [1]:
workspace = 'local'   # set to udacity if running in the workspace
version = 'One'
env_file_name = f'./env/version{version}/Reacher.app'
if workspace == 'udacity':
    !pip -q install ./python
    if version == 'One':
        env_file_name = f'/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'
    else:
        env_file_name = f'/data/Reacher_Linux_NoVis/Reacher.x86_64'

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name=env_file_name)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Train the agent in the Environment

In [5]:
import numpy as np
from collections import namedtuple, deque
from datetime import datetime

import random
import torch
from replaybuffer import ReplayBuffer
from ddpg.agent import Agent as DDPG_Agent

import matplotlib.pyplot as plt
%matplotlib inline

config = { 
    "state_size": state_size,    # dimension of each state
    "action_size": action_size,  # dimension of each action
    "buffer_size": int(1e6),     # replay buffer size
    "batch_size": 64,            # minibatch size
    "random_seed": None,         # random seed
    "gamma": 0.99,               # discount factor
    "tau": 1e-3,                 # for soft update of target parameters
    "lr_actor": 1e-4,            # learning rate of the actor 
    "lr_critic": 1e-3,           # learning rate of the critic
    "weight_decay": 0,           # L2 weight decay
    "learn_every": 50,           # learn from replay buffer every time step
    "learn_batch_size": 10,      # number of batches to learn from replay buffer every learn_every time step
    "grad_clip": 1.0,            # gradient value to clip at for critic
    "eps_start": 1.0,            # starting value of epsilon, for epsilon-greedy action selection
    "eps_end": 0,                # minimum value of epsilon
    "eps_decay": 1e-5,           # multiplicative factor (per episode) for decreasing epsilon
    "print_every": 20,            # Print average every x episode,
    "episode_steps": 1000
}

def ddpg(agent, n_episodes=1000):
    mean_scores = []                               # list of mean scores from each episode
    last_100_avgs = []                             # moving average tracker of mean of last 100 episodes
    scores_window = deque(maxlen=100)              # mean scores from most recent 100 episodes
    best_scores = 0
    print(f'Starting training {datetime.now()}')
    for i_episode in range(1, n_episodes+1):
        start_datetime = datetime.now()
        env_info = env.reset(train_mode=True)[brain_name] 
        states = env_info.vector_observations 
        scores = np.zeros(num_agents)
        
        agent.reset()
        for t in range(config["episode_steps"]):
            print(f'\rEpisode {i_episode}\tTimestep: {t}\tScore: {scores}\tBest Episode Score: {best_scores}', end='\r')

            actions = agent.act(states, add_noise=True)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished

            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done, t, np.mean(scores))     
            
            states = next_states
            scores += rewards
            if np.any(dones):
                print(f'\rEpisode {i_episode}\tDone at step {t}')
                break
                
        episode_scores = np.mean(scores)
        if episode_scores > best_scores:
            best_scores = episode_scores
            agent.learn_episode()

        mean_scores.append(np.mean(scores))             # save mean score for the episode
        scores_window.append(mean_scores[-1])           # save mean score to window
        last_100_avgs.append(np.mean(scores_window))    # save moving average
        if i_episode % config["print_every"] == 0:
            print(f'\rEpisode {i_episode}\tLast 100 Episodes Moving Avg: {last_100_avgs[-1]:.1f} took {datetime.now() - start_datetime}')
                  
        if last_100_avgs[-1] >= 30.0 and i_episode >= 100:
            print(f'\nEnvironment solved in {i_episode-100:d} episodes!\tAverage Score: { last_100_avgs[-1]:.2f}') 
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
            
    return mean_scores, last_100_avgs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
memory = ReplayBuffer(device=device, config=config)
agent = DDPG_Agent(memory=memory, device=device, config=config)
scores, avgs = ddpg(agent, n_episodes=1000)    # The maximum step before the episode will end is 1,000

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='DDPG')
plt.plot(np.arange(len(scores)), avgs, c='r', label='Moving AVG')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Starting training 2019-11-11 20:16:08.338441


KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 